## CCNSS 2018 Module 5: Whole-Brain Dynamics and Cognition
# Tutorial 4: Kuramoto model (II)


*Please execute the cell bellow in order to initialize the notebook environment*

In [0]:
!if [ ! -d data ]; then git clone https://github.com/ccnss/ccnss2018_students; \
                        cp -rf ccnss2018_students/module5/4_kuramoto model_2/data ./; fi

In [0]:
import matplotlib.pyplot as plt    # import matplotlib
import numpy as np                 # import numpy
import math                        # import basic math functions
import random                      # import basic random number generator functions
import csv                         # import CSV(Comma Separated Values) file reading and writing
import scipy as sp                 # import scipy
from scipy import sparse           # import sparse module from scipy
from scipy import signal           # import signal module from scipy

import os                          # import basic os functions
import time                        # import time to measure real time
import collections                 # import collections
import networkx as nx              # import networkx 


data_folder = 'data'

print('Available data files:\n'+'\n'.join(sorted(os.listdir(data_folder))))

data_file_1 = os.path.join(data_folder, 'gong_net_list.txt')
data_file_2 = os.path.join(data_folder, 'macaque_net_list.txt')

In [0]:
net_gong_ut = [ row for row in csv.reader(open(data_file_1,'r'),delimiter='\t')]
net_gong_ut = np.array(net_gong_ut).astype(int)
net_gong_lt = net_gong_ut[:,(1,0,2)]
net_gong = np.concatenate( (net_gong_ut, net_gong_lt), axis=0)

net_row_gong = net_gong[:,0]-1
net_column_gong = net_gong[:,1]-1
net_value_gong = net_gong[:,2]
net_coord_gong = np.column_stack((net_row_gong,net_column_gong))


# Objectives

In this notebook, we add time-delay to the Kuramoto model, to account for the delay in transfer between brain areas. We then compute the phase-lead/lag realtionship between each pair of oscillators, which may give us clues on the information transfer between different areas of the brain. Next, we will construct functional connectivity from a Kuramoto model on network. We can compare the resulting functional connectivity with the underlying structural network. 


## Background

1. There is inherent time delay in information transfer between different areas of the brain, which will likely be proportional to the length of the neural pathways (ex: fiber bundles) connecting different areas. We will add such time delay to our Kuramoto model. The time-delay Kuramoto model exhibits stable patterns of phase-lead/lag relationships between nodes, which may give us clues on the information flow pattern of the brain network.

2. Kuramoto model on a network generates a system of multiple time series. We can naturally construct a functional connectivity from the time series. This functional connectivity will resemble underlying structural connectivity (network structure) when the coupling strength $K$ of the system has a critical value.


**EXERCISE 1: Kuramoto model with time delay and phase delay, and equivalence between them ** 

To make Kuramoto model account for the delays between the nodes, we can add a time delay term to the model.

$$ \frac {d \theta_i} {dt} = \omega_i + \frac{K}{N} \sum_{j=1}^{N} A_{ij} \sin( \theta_j(t-\tau) - \theta_i(t)),  \quad    i=1,...N. $$

If the delay is short enough (less than the period of the oscillation), the time delay simply will act as a phase delay term: 

$$ \frac {d \theta_i} {dt} = \omega_i + \frac{K}{N} \sum_{j=1}^{N} A_{ij} \sin( \theta_j(t) - \beta - \theta_i(t)),  \quad    i=1,...N. $$

• Make codes for above equations.


• Then repeat the exercises from previous tutorial with above time-delay equation with $ \tau = 10 ms $ and  phase-delay equation with $ \beta = 2\pi/10 $: using a Gong network of human brain, compute, plot, and print $ \langle r \rangle _t $ for $ K\in\{0, 3, ..., 33\} $ for each equations. Remember, to compare the results from above two equations, one must use same initial conditions for the $\theta_i $.

• Generate a time series $ \theta_i $ from the models with sufficiently large $ K $, such that  $ \langle r \rangle _t $  is high enough but not close to one (larger than 0.4 but less than 0.8: $ K $ ~ 30 would be a good candidate number to start guessing).

In [0]:
# code for computing r
def compute_r(theta):

    len_t = theta.shape[0]
    len_x = theta.shape[1]
    r_t_all = 1/len_x*np.absolute(np.sum( math.cos(theta) +  math.sin(theta)*1j ) , axis=1 )
    t_t_avg = np.mean(r_t_all)
                                 
    psi = np.angle( ( math.cos(theta_1[k+1])+math.cos(theta_2[k+1]) ) + ( math.sin(theta_1[k+1])+ math.sin(theta_2[k+1]) )*1j )



In [0]:
# code for time averaging r   
def compute_r_tavg(r):

    r_half = r[int(0.5*len(r)):len(r)-1]
    r_tavg = np.mean(r_half)
    return r_tavg

In [0]:
#insert your code here for the time delayed Kuramoto model

# time delayed kuramoto model code for general network
def simulate_k_mat_time_delay(N,K,W,tau,theta_ini,t,net_coord,net_row,net_column):
    
    dt = t[1]-t[0]
    theta = np.zeros((len(t),N))
    r = np.zeros(len(t))
    psi = np.zeros(len(t))
    
    theta[0,:] = theta_ini
    r[0] = 1/N*np.absolute( np.sum(np.cos(theta[0,:]))  + np.sum(np.sin(theta[0,:]))*1j )
    psi[0] = np.angle(  np.sum(np.cos(theta[0,:])) + np.sum(np.sin(theta[0,:]))*1j )
    
    for k in range(0,tau):
        theta[k+1,:] = theta[k,:] + dt*( W )
        r[k+1] = 1/N*np.absolute( np.sum(np.cos(theta[k+1,:]))  + np.sum(np.sin(theta[k+1,:]))*1j )
        psi[k+1] = np.angle(  np.sum(np.cos(theta[k+1,:])) + np.sum(np.sin(theta[k+1,:]))*1j )
              
    for k in range(tau,len(t)-1):
        
        mat_size =  np.amax(net_coord)+1
        term_sin = np.sin( theta[k-tau,net_row] - theta[k,net_column] )
        term_mat = sp.sparse.coo_matrix((term_sin, (net_row,net_column)), shape=(mat_size,mat_size))
        term_mat_csc = sp.sparse.csc_matrix(term_mat)
        term_sum = term_mat_csc.sum(axis=0)

        theta[k+1,:] = theta[k,:] + dt*( W + K* term_sum )
        r[k+1] = 1/N*np.absolute( np.sum(np.cos(theta[k+1,:]))  + np.sum(np.sin(theta[k+1,:]))*1j )
        psi[k+1] = np.angle(  np.sum(np.cos(theta[k+1,:])) + np.sum(np.sin(theta[k+1,:]))*1j )
           
    return theta,r,psi

In [0]:
#insert your code here for the phase delayed Kuramoto model

# phase delayed kuramoto model code for general network
def simulate_k_mat_phase_delay(N,K,W,beta,theta_ini,t,net_coord,net_row,net_column):
    
    dt = t[1]-t[0]
    theta = np.zeros((len(t),N))
    r = np.zeros(len(t))
    psi = np.zeros(len(t))
    
    theta[0,:] = theta_ini
    r[0] = 1/N*np.absolute( np.sum(np.cos(theta[0,:]))  + np.sum(np.sin(theta[0,:]))*1j )
    psi[0] = np.angle(  np.sum(np.cos(theta[0,:])) + np.sum(np.sin(theta[0,:]))*1j )
        
    for k in range(len(t)-1):
        
        # finish your code by filling in here
        
        term_sum = 

        theta[k+1,:] = theta[k,:] + dt*( W + K* term_sum )
        r[k+1] = 1/N*np.absolute( np.sum(np.cos(theta[k+1,:]))  + np.sum(np.sin(theta[k+1,:]))*1j )
        psi[k+1] = np.angle(  np.sum(np.cos(theta[k+1,:])) + np.sum(np.sin(theta[k+1,:]))*1j )
           
    return theta,r,psi

In [0]:
# code for comparison of two types of model (time delay and phase delay) on Gong network

# number of oscillators
N_gong = 78

# time delay
tau = 10 # in units of dt

# phase delay
beta = 2*np.pi/10

# initial frequency
W = 10*2*np.pi

# time
dt = 0.001
t = np.arange(0,10,dt)

# initial theta
random.seed()

theta_ini = np.random.rand(N_gong)*2*np.pi
theta_ini[1] = theta_ini[0] + 0.5*np.pi
theta_ini[2] = theta_ini[0] -0.5*np.pi

Ks = np.arange(0,22,2)
rs_pd = np.zeros(len(Ks))
rs_td = np.zeros(len(Ks))

theta_ini_td=theta_ini
theta_ini_pd=theta_ini

for k in range(len(Ks)):    
    # what time is it now?
    rt0 = time.time()
    
    # core code 
    theta_td,r_td,psi_td = simulate_k_mat_time_delay(N_gong,Ks[k]*(1/N_gong),W,tau,theta_ini_td,t,net_coord_gong,net_row_gong,net_column_gong)
    rs_td[k] = compute_r_tavg(r_td)
    theta_ini_td
    
    theta_pd,r_pd,psi_pd = simulate_k_mat_phase_delay(N_gong,Ks[k]*(1/N_gong),W,beta,theta_ini_pd,t,net_coord_gong,net_row_gong,net_column_gong)
    rs_pd[k] = compute_r_tavg(r_pd)
    theta_ini_pd

    # what time is it now and how long did it take?
    rt1 = time.time()
    print('time spent for calculation:{}s'.format(rt1-rt0))

In [0]:
# plot

plt.figure()
plt.plot(Ks,rs_td,'C0s')
plt.plot(Ks,rs_pd,'rd')
plt.xlabel('K')
plt.ylabel('< r >')
plt.title('Gong network')
plt.show()

print("r from time delay K-model:", rs_td)
print("r from phase delay K-model:", rs_pd)

** EXPECTED OUTPUT **

![](https://github.com/ccnss/ccnss2018_students/raw/master/module5/4_kuramoto_model_2/figures/1_pd_vs_tp.png)


```
r from time delay K-model: [0.11049591 0.1577925  0.24157359 0.28125095 0.29806161 0.31194672
 0.33116812 0.36152939 0.39519595 0.42499737 0.45220356]
r from phase delay K-model: [0.11049591 0.15785225 0.24187163 0.28134417 0.29959361 0.31640776
 0.33795266 0.36941168 0.40366517 0.43349274 0.46021181]
```



In [0]:
# code to generate two types of Kuramoto model with sufficiently large coupling strength of 30

K_lg= 30

theta_td_lg,r_td_lg,psi_td_lg = simulate_k_mat_time_delay(N_gong,K_lg*(1/N_gong),W,tau,theta_ini,t,net_coord_gong,net_row_gong,net_column_gong)
rs_td_lg= compute_r_tavg(r_td_lg)
print("r from time delay K-model:", rs_td_lg)

theta_pd_lg,r_pd_lg,psi_pd_lg = simulate_k_mat_phase_delay(N_gong,K_lg*(1/N_gong),W,beta,theta_ini,t,net_coord_gong,net_row_gong,net_column_gong)
rs_pd_lg= compute_r_tavg(r_pd_lg)
print("r from phase delay K-model:", rs_pd_lg)

** EXPECTED OUTPUT **





```
r from time delay K-model: 0.4966947824741849
r from phase delay K-model: 0.5035572264579842
```



**EXERCISE 2: Phase difference between oscillators, and *corr* between degree and phase of each node **



we can calculate phase differences $\Delta \theta$ between each pair of oscillators:

$$ \Delta \theta_{ij} = \theta_i - \theta_j .$$

Or, we can rewrite $\Delta \theta$ in another way:

$$ \Delta \theta_{ij} = \arg( e^{i (\theta_i - \theta_j)} ),$$

which means we take the angle of $ e^{i (\theta_i - \theta_j)} $ from the real axis. We will follow the second representation for our computation.

This phase difference may give a valuable information about the "directionality" between pair of oscillators.

• First, from the phase-delay (or time-delay) time series we generated, calculate the $\theta_{ij}$ between all pairs of time series, and build a phase-difference matrix. The resulting matrix will be anti-symmetric. Use only the latter half of time series in the process.

• From the phase-difference matrix, we can compute the average phase-difference for each node. Calculate the row-sum of the matrix:

$$ \theta_i = 1/N \sum_{j=1}^{N} \theta_{ij},$$

then we can have a vector of averaged phase-differences, each element of the vector corresponding for each node.

• Finally, do a scatterplot of the average phase-difference and degree for each node. Is there any pattern between these two quantities? Calculate the Pearson correlation coefficient between these two vectors.


In [0]:
# insert your code for calculating phase difference

def phase_diff_mat(theta):
 
    # theta must has dimension TxN, where T is the length of time points and N is the number of nodes
    
    N_len = theta.shape[1]
        
    PDiff_mat= np.zeros((N_len,N_len))    
    for ch1 in range(N_len):
        for ch2 in range(ch1+1,N_len):
            PDiff=theta[:,ch1]-theta[:,ch2]
            PDiff_exp = np.angle( np.exp(1j*PDiff) )
            PDiff_exp_mean = np.mean(PDiff_exp)
            PDiff_mat[ch1,ch2] = PDiff_exp_mean
            PDiff_mat[ch2,ch1] = -1*PDiff_exp_mean
            
    PDiff_mean = np.mean(PDiff_mat,axis=1)        
    
    #alternative code
    #arr = np.array([np.roll(theta, i, axis=1) for i in range(N_len)])
    #PDiff_mat = theta[None, :] - arr
    #PDiff_mean = PDiff_mat.mean(1)
    
    return PDiff_mean,PDiff_mat

In [0]:
# calculate the phase difference 

theta_pd_mean,theta_pd_mat = phase_diff_mat(theta_pd[-int(theta_pd.shape[0]/2):,:])

# calculate the degree of each node of gong network 

net_gong_size =  np.amax(net_coord_gong)+1
net_gong_mat = sp.sparse.coo_matrix((net_value_gong, (net_row_gong,net_column_gong)), shape=(net_gong_size,net_gong_size) )
net_gong_mat_csc = sp.sparse.csc_matrix(net_gong_mat)
net_gong_degree = np.sum(net_gong_mat_csc,axis=0).T

# plot
plt.figure()
plt.plot(net_gong_degree,theta_pd_mean,'C0s')
plt.xlabel('K')
plt.ylabel('theta')
plt.title('theta vs K')
plt.show()

# corr
corr_degree_phase = np.corrcoef(net_gong_degree,theta_pd_mean)
print("corr. of degree and phase of each node:", corr_degree_phase[0,1])

**EXPECTED OUTPUT**

![](https://github.com/ccnss/ccnss2018_students/raw/master/module5/4_kuramoto_model_2/figures/2_corr_degree_phase.png)


```
corr. of degree and phase of each node: -0.5463676447067697
```



**EXERCISE 3 (optional, possible for mini-project) : constructing functional connectivity from the oscillators via calculating $ PLI $ between them**

We begin with the previous phase-delayed Kuramoto equations:

$$ \frac {d \theta_i} {dt} = \omega_i + \frac{K}{N} \sum_{j=1}^{N} A_{ji} \sin( \theta_j - \theta_i - \beta),  \quad    i=1,...N, $$

where $A_{ji}$ is the connectivity between two nodes $i$ and $j$ (with value of either 1 or 0).*italicized text*

• Using a Gong network of human brain, give initial phase $\theta_i$ randomly, and let coupling strength $K$ vary from 0.1 to 1 by increment of 0.1.

• At each coupling strength, compute the average $ \langle r \rangle _t $ (choose only the latter-half of time series).

• At each coupling strength, measure $ PLI_{ij} $ between the time series. Again, choose only the latter-half of time series. Ultimately, construct a $ PLI $ matrix containing all the $ PLI_{ij} $ between possible pairs. The resulting matrix will be symmetric. 

• Compute the average $ PLI $ for each strength $K$ (sum of all $ PLI_{ij} $s of the $ PLI $ matrix).  Plot the average $ \langle r \rangle _t $ and average $ PLI $ for each coupling strength $K$. What kind of patterns do you observe from the changes of  $ \langle r \rangle _t $  and  average $ PLI $ with respect to the changes in $K$?

**EXERCISE 4 (optional, possible for mini-project) : Calculating $corr$ between structural connectivity network and functional connectivity network**

With the $ PLI $ matrix we built in exercise 3, we can compare against structural connectivity.

• First, give a threshold of 0.3, and binarize the connectivity matrix. If the matrix element is larger than 0.3, it will turn into value of 1, otherwise 0. In the end, we will have one connectivity matrix per one value of coupling strength $ K $.

• After constructing the connectivity matrix, compute the degree for each node. In the due process, we will have a vector consisting of the degree for each node, per one value of coupling strength $ K $.

• For Gong network, we can also compute a vector of degrees for each node.

• Finally, compute the Pearson correlation $ c $ between the degree vector from the functional connecvtivity, and the degree vector from Gong network. In the graph of Exercise 4, add a plot of the correalation $ c $ as a function of time. At which coupling strength $ K $ does the correlation $ c $ has largest value? At this $ K $, what is the value of $ r(t) $ and average $ corr $? We will call this coupling strength the critical strength $ Kc $.




**EXERCISE 5 : Runge-Kutta method (optional, possible for mini-project)**

Remerber in the first module how we learned to compute differential equations via Euler method? In the Wilson-Cowan model, we discretized $dt$ into $\Delta t$. To requote:

\begin{align}
\frac{dE}{dt} = f(E,I,t) \to \frac{E(t+\Delta t)- E(t)}{\Delta t}&= f(E,I,t) \hspace{5 mm}\text{ and }\hspace{5mm} \\ \frac{dI}{dt} = g(E,I,t) \to \frac{I(t+\Delta t)-I(t)}{\Delta t} &=g(E,I,t),\\
\end{align}

henceforth equating the future trajectory of $E(t)$  and $I(t)$ in the following way:

\begin{align}
E(t+\Delta t) = &E(t) + \Delta t f(E,I,t) \hspace{5 mm}\text{ and }\hspace{5mm}\ \\
I(t+\Delta t) = &I(t) + \Delta t g(E,I,t). \\
\end{align}


 Euler method has served us well, but in order to simulate more complex differential equations, more refined method is needed.
 
 If we inspect Euler method closely, there is a inherent asymmetry in the right side of the equation. We only evaluate the value of $f(E,I,t)$ or $g(E,I,t)$ at time $t$. Therefore, we are predicting the (future) value of $E(t+\Delta t)$ solely based on the values at the current moment, $t$.

Based on this observation, we can generalize Euler method. Here we introduce (2nd order) Runge-Kutta method, which is exactly a generalization of Euler method towards resolving this asymmetry. With Runge-Kutta method, we begin with writing Wilson-Cowan equation as the following:

$$ \frac{E(t+\Delta t)- E(t)}{\Delta t} = \frac{1}{2} \{ f(E(t),I(t),t)+ f(E(t+\Delta t), I(t+\Delta t),t+\Delta t) \},$$

Now, as an approximation, we write:

\begin{align}
\tilde E(t+\Delta t) &= E(t) + \Delta t f(E(t),I(t),t) \\
E(t+\Delta t) &= E(t) + \frac{\Delta t}{2} \{f(E(t),I(t),t) +f(\tilde E(t+\Delta t),\tilde I(t+\Delta t),t+\Delta t) \}, \\
\end{align}

and similarly for the $I$.


• Write an expression of Runge-Kutta method applied to the following Kuramoto model with two nodes:

$$ \frac {d \theta_1(t)} {dt} = \omega_1 + \frac{K}{N} \sin( \theta_2(t) - \theta_1(t)),  \quad  \\  \frac {d \theta_2(t)} {dt} = \omega_2 + \frac{K}{N} \sin( \theta_1(t) - \theta_2(t)) .$$

• Write the code for above equation. The way one wants to do it, is to define a function for R-K2 method and put $f(t)$ as an input to the function. Compare the result is similar with the Euler method of Tutorial #3, exercise 2.





In [0]:
def euler( f, x0, t ):
    """ AUTHOR:
        Jonathan Senning <jonathan.senning@gordon.edu>
        Gordon College
        Based Octave functions written in the spring of 1999
        Python version: March 2008, October 2008
    """
    """Euler's method to solve x' = f(x,t) with x(t[0]) = x0.

    USAGE:
        x = euler(f, x0, t)

    INPUT:
        f     - function of x and t equal to dx/dt.  x may be multivalued,
                in which case it should a list or a NumPy array.  In this
                case f must return a NumPy array with the same dimension
                as x.
        x0    - the initial condition(s).  Specifies the value of x when
                t = t[0].  Can be either a scalar or a list or NumPy array
                if a system of equations is being solved.
        t     - list or NumPy array of t values to compute solution at.
                t[0] is the the initial condition point, and the difference
                h=t[i+1]-t[i] determines the step size h.

    OUTPUT:
        x     - NumPy array containing solution values corresponding to each
                entry in t array.  If a system is being solved, x will be
                an array of arrays.
    """

    n = len( t )
    x = np.array( [x0] * n )
    for i in range( n - 1 ):
        x[i+1] = x[i] + ( t[i+1] - t[i] ) * f( x[i], t[i] )

    return x

In [0]:
# new code for 2nd order Runge-Kutta method

def rk2( f, x0, t ):
    """ AUTHOR:
        Jonathan Senning <jonathan.senning@gordon.edu>
        Gordon College
        Based Octave functions written in the spring of 1999
        Python version: March 2008, October 2008
    """
    """Heun's method to solve x' = f(x,t) with x(t[0]) = x0.

    USAGE:
        x = rk2(f, x0, t)

    INPUT:
        f     - function of x and t equal to dx/dt.  x may be multivalued,
                in which case it should a list or a NumPy array.  In this
                case f must return a NumPy array with the same dimension
                as x.
        x0    - the initial condition(s).  Specifies the value of x when
                t = t[0].  Can be either a scalar or a list or NumPy array
                if a system of equations is being solved.
        t     - list or NumPy array of t values to compute solution at.
                t[0] is the the initial condition point, and the difference
                h=t[i+1]-t[i] determines the step size h.

    OUTPUT:
        x     - NumPy array containing solution values corresponding to each
                entry in t array.  If a system is being solved, x will be
                an array of arrays.
    """
    n = len( t )
    x = np.array( [x0] * n )
    #print(x.shape)
    for i in range( n - 1 ):
        h = t[i+1] - t[i]
        k1 = h * f( x[i], t[i] )
        k2 = h * f( x[i] + k1, t[i+1] )
        x[i+1] = x[i] + ( k1 + k2 ) / 2.0

    return x

In [0]:
def f( x, t ):
        return x * np.sin( t )

a, b = (0.0, 15.0)
x0 = -1.0

n = 151
t = np.linspace( a, b, n )
h = t[1] - t[0];

# compute various numerical solutions
x_euler = euler( f, x0, t )
x_heun = rk2( f, x0, t )

# compute true solution values in equal spaced and unequally spaced cases
x = -np.exp( 1.0 - np.cos( t ) )

plt.figure()
#    figure( 1 )
plt.subplot( 2, 1, 1 )
plt.plot( t, x_euler, 'b-o', t, x_heun, 'g-o' ,t, x, 'r-*'  )
plt.xlabel( '$t$' )
plt.ylabel( '$x$' )
plt.title( 'Solutions of $dx/dt = x \sin t$, $x(0)=-1$ ($h = %4.2f$)' % h )
plt.legend( ( 'Euler', 'Heun' ,'true'), loc='lower left' )

#    figure( 2 )
plt.subplot( 2, 1, 2 )
plt.plot( t, (x_euler - x), 'b-o', t, (x_heun - x), 'g-o' )
plt.xlabel( '$t$' )
plt.ylabel( '$x - x^*$' )
plt.title( 'Errors in sol. of $dx/dt = x \sin t$, $x(0)=-1$ ($h = %4.2f$)' % h ) 
plt.legend( ( 'Euler', 'Heun' ), loc='upper left' )

plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
plt.show()